1. Import Libraries 

In [1]:
import tensorflow as tf
import numpy as np

import cv2    ##for image processing
from PIL import Image

from tensorflow.keras import Model

2. Image preprocessing:

In [2]:
content_image = cv2.resize(cv2.imread('C:/Users/pjvya/neckarfront.jpg'), (300, 300))
style_image = cv2.resize(cv2.imread('C:/Users/pjvya/style_images/Starry_night.jpg'), (300, 300))

content_image = tf.image.convert_image_dtype(content_image, tf.float32)# brings image pixels to 0 and 1
style_image = tf.image.convert_image_dtype(style_image, tf.float32)

content_image = cv2.cvtColor(np.array(content_image), cv2.COLOR_BGR2RGB)
style_image = cv2.cvtColor(np.array(style_image), cv2.COLOR_BGR2RGB)


3. Loss Function

In [3]:
def loss_fun(style_outputs, content_outputs, style_target, content_target):
    
    # Define the weights for content and style losses
    style_weight = 0.1
    content_weight = 100
    
    # Calculate the content loss
    content_loss = content_weight * tf.reduce_mean(tf.square(content_outputs - content_target))
    
    # Calculate the style loss using the Gram matrix outputs and target Gram matrices
    style_losses = []
        # Iterate over pairs of style outputs and style targets
    for output_, target_ in zip(style_outputs, style_target):
        # Calculate the mean squared difference between output and target
        individual_loss = tf.reduce_mean(tf.square(output_ - target_))
    
        # Append the individual style loss to the list
        style_losses.append(individual_loss)

    # Sum up all the individual style losses to obtain the final style loss
    style_loss = style_weight * tf.add_n(style_losses)
    
    # Combine content and style losses using the defined weights
    total_loss = content_loss + style_loss
    #print(total_loss)
    return total_loss

4. Gram Matrix

In [4]:
def gram_matrix(input_tensor):
    # Reshape the input tensor to have a shape of [-1, channels]  
    channels = int(input_tensor.shape[-1])    
    a = tf.reshape(input_tensor, [-1, channels]) 
    #print (a)
    n = tf.shape(a)[0]
    # Calculate the Gram matrix by multiplying the reshaped tensor with its transpose
    gram = tf.matmul(a, a, transpose_a=True)
    # Normalize the matrix by dividing by the total number of elements in the reshaped tensor
    return gram / tf.cast(n, tf.float32)

5. Define Model

In [5]:
def load_vgg(input_shape):
    
    # Load the VGG19 model with specified input shape
    vgg = tf.keras.applications.VGG19(include_top=False, weights="imagenet", input_shape=input_shape)
    
    # Set the model to be non-trainable
    vgg.trainable = False
    
    # Define the content and style layer names
    content_layers = ['block4_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    
    # Get the output tensor of the content layer
    content_layer = content_layers[0]
    content_output = vgg.get_layer(content_layer).output 
    #print(content_output)
    # Get the output tensors of the style layers
    # Initialize an empty list to store style outputs
    style_outputs = []

    # Iterate through the style layers and get their respective outputs
    for style_layer in style_layers:
        style_layer_output = vgg.get_layer(style_layer).output
        style_outputs.append(style_layer_output)

    # Assign the list of style outputs to the style_output variable
    style_output = style_outputs
    #print (style_output)    
    # Compute the Gram matrices for each style layer's output
    # Initialize an empty list to store Gram matrices of style outputs
    gramMatrix_style_outputs = []

    # Iterate through the style outputs and calculate their Gram matrices
    for output_ in style_output:
        gramMatrix_style_output = gram_matrix(output_)
        gramMatrix_style_outputs.append(gramMatrix_style_output)

    # Assign the list of Gram matrices to the gram_style_output variable
    gramMatrix_style_output = gramMatrix_style_outputs
    #print(gramMatrix_style_output)
    # Create a new model that takes the VGG input and outputs content and style features
    # Define the inputs of the model
    model_inputs = [vgg.input]

    # Define the outputs of the model as a list containing content_output and gramMatrix_style_output
    model_outputs = [content_output, gramMatrix_style_output]

    # Create the model
    model = Model(inputs=model_inputs, outputs=model_outputs)
    
    return model

6. Load Model

In [6]:
vgg_model = load_vgg((300, 300, 3))
content_target = vgg_model(np.array([content_image*255]))[0]
style_target = vgg_model(np.array([style_image*255]))[1]

7. Train the Model

In [7]:
import time

#optimizer is to minimize the total loss function by updating the generated image.
opt = tf.optimizers.Adam(learning_rate=0.001, beta_1=0.9,beta_2=0.999, epsilon=1e-8)
losses = []
epochs = []
def train_step(image, epoch):
    total_time = 0
    num_iterations = 1

    start = time.time()
    with tf.GradientTape() as tape:
        output = vgg_model(image*255) # call model again: forward pass, 
        #image is in the range btw 0 and 1 --> multiply with 255 to give the value back
        loss = loss_fun(output[1], output[0], style_target, content_target)
    gradient = tape.gradient(loss, image)
    opt.apply_gradients([(gradient, image)])# optimize gredient and backpropagate image
    # cliping btw 0 and 1 so value of output will not be more than 255
    image.assign(tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0))
    
    stop = time.time()
    total_time += (stop - start)
    num_iterations += 1
    if epoch % 100 ==0:
        avg_time = total_time / num_iterations
        tf.print(f"Loss = {loss} time = {avg_time} seconds")
        losses.append(loss)
        epochs.append(epoch)
    

8. Import image data

In [8]:
EPOCHS = 10
image = tf.image.convert_image_dtype(content_image, tf.float32)
image_list = [image]
image = tf.Variable(image_list) # make all pixels of image changable, [] to make batch
for i in range(EPOCHS):
    train_step(image, i)

Loss = 15113010176.0 time = 2.67716121673584 seconds


9. Deprocessesing image

In [9]:
tensor = tf.clip_by_value(image * 255, 0.0, 255.0)
tensor = np.array(tensor, dtype=np.uint8)
if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
generated_image = cv2.cvtColor(tensor, cv2.COLOR_RGB2BGR)
output_path = 'C:/Users/pjvya/generated_images/10iterations.jpg'
cv2.imwrite(output_path,generated_image)


True